In [116]:
# Importing Libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np 
from sklearn.preprocessing import MinMaxScaler
# from datetime import datetime, timedelta

In [117]:
headers = {'Accept-Language': 'en-US,en;q=0.8'}
imdb_page = requests.get("https://www.imdb.com/list/ls009609925/", headers=headers)
#imdb_filming_dates = requests.get("https://www.imdb.com/title/tt0172495/locations?ref_=ttloc_ql_5")
#imdb_page = requests.get("https://www.imdb.com/list/ls031758648/")
soup = BeautifulSoup(imdb_page.content, "html.parser")

In [118]:
item_content = soup.find_all('div', class_="lister-item-content")

In [37]:
#item_content[0].find_all('p', class_="text-muted text-small")[1].text.split("|")[1].replace("\n", "").split(":")[1]
#item_content[0].find('p', class_="").text.replace("\n", "")
#item_content[0].find_all('h3', class_="lister-item-header")[0].text.replace('\n', "")
#item_content[0].find_all('img', class_="alt")
#item_content[2].find('a')
#item_content[2].find('span', class_='lister-item-year').text.replace("(","").replace(")","")
#item_content[0].find('span', class_="ipl-rating-star__rating").text
#item_content[0].find('span', class_="runtime").text
#item_content[0].find('span', class_="genre").text.replace("\n","")

In [119]:
unique_ref = []
for index in range(len(item_content)):
     unique_ref.append(str(item_content[index].find('a')).split("/")[2])

In [120]:
movie_names = []
release_date = []
description = []
directors = []
movie_stars = []
ratings = []
duration = []
genre = []

for index in range(len(item_content)):
    movie_names.append(item_content[index].find('a').text)
    release_date.append(item_content[index].find('span', class_='lister-item-year').text.replace("(","").replace(")",""))
    description.append(item_content[index].find('p', class_="").text.replace("\n", ""))
    directors.append(item_content[index].find_all('p', class_="text-muted text-small")[1].
                     text.split("|")[0].replace("\n", "").split(":")[1])
    movie_stars.append(item_content[index].find_all('p', class_="text-muted text-small")[1]
                       .text.split("|")[1].replace("\n", "").split(":")[1])
    ratings.append(item_content[index].find('span', class_="ipl-rating-star__rating").text)
    duration.append(item_content[index].find('span', class_="runtime").text)
    genre.append(item_content[index].find('span', class_="genre").text.replace("\n","").strip())

In [121]:
release_date = ' '.join(release_date).replace('I','').split()

In [122]:
# Filming Dates
#https://www.imdb.com/title/tt0172495/locations?ref_=ttloc_ql_5
filming_dates = []
for index in range(len(unique_ref)):
    imdb_page = requests.get("https://www.imdb.com/title/"+unique_ref[index]+"/locations?ref_=ttco_ql_5", headers=headers)
    soup = BeautifulSoup(imdb_page.content, "html.parser")
    item_content = soup.find_all('li', class_="ipl-zebra-list__item")
    filming_dates.append(item_content[0].text.replace("\n", "").strip())

In [143]:
# Converting into DataFrames

imdb_df = pd.DataFrame(list(zip(movie_names, release_date, directors, duration, ratings, genre, movie_stars, filming_dates, description)),
               columns =['Movie Name', 'Date Release', "Director", "Duration (Mins)", "Ratings", "Genre", "Movie Stars", "Filming Dates", "Description"])
#pd.set_option('display.max_columns', 8)
imdb_df

,Movie Name,Date Release,Director,Duration (Mins),Ratings,Genre,Movie Stars,Filming Dates,Description
0,Gladiator,2000,Ridley Scott,155 min,8.5,"Action, Adventure, Drama","Russell Crowe, Joaquin Phoenix, Connie Nielsen...",18 January 1999 - 29 May 1999,A former Roman General sets out to exact venge...
1,Inception,2010,Christopher Nolan,148 min,8.8,"Action, Adventure, Sci-Fi","Leonardo DiCaprio, Joseph Gordon-Levitt, Ellio...",19 June 2009 - 22 November 2009,A thief who steals corporate secrets through t...
2,The Lord of the Rings: The Fellowship of the Ring,2001,Peter Jackson,178 min,8.8,"Action, Adventure, Drama","Elijah Wood, Ian McKellen, Orlando Bloom, Sean...",11 October 1999 - 22 December 2000,A meek Hobbit from the Shire and eight compani...
3,The Lord of the Rings: The Two Towers,2002,Peter Jackson,179 min,8.7,"Action, Adventure, Drama","Elijah Wood, Ian McKellen, Viggo Mortensen, Or...",11 October 1999 - 22 December 2000,While Frodo and Sam edge closer to Mordor with...
4,The Lord of the Rings: The Return of the King,2003,Peter Jackson,201 min,8.9,"Action, Adventure, Drama","Elijah Wood, Viggo Mortensen, Ian McKellen, Or...",11 October 1999 - 22 December 2000,Gandalf and Aragorn lead the World of Men agai...
...,...,...,...,...,...,...,...,...,...
95,Zombieland,2009,Ruben Fleischer,88 min,7.6,"Adventure, Comedy, Fantasy","Jesse Eisenberg, Emma Stone, Woody Harrelson, ...",February 2009 - April 2009,A shy student trying to reach his family in Oh...
96,Speed,1994,Jan de Bont,116 min,7.2,"Action, Adventure, Thriller","Keanu Reeves, Dennis Hopper, Sandra Bullock, J...",7 September 1993 - 23 December 1993,A young police officer must prevent a bomb exp...
97,Jumanji,1995,Joe Johnston,104 min,7,"Adventure, Comedy, Family","Robin Williams, Kirsten Dunst, Bonnie Hunt, Jo...",14 November 1994 - 7 April 1995,When two kids find and play a magical board ga...
98,"The Chronicles of Narnia: The Lion, the Witch ...",2005,Andrew Adamson,143 min,6.9,"Adventure, Family, Fantasy","Tilda Swinton, Georgie Henley, William Moseley...",28 June 2004,Four kids travel through a wardrobe to the lan...


In [144]:
imdb_df['Duration (Mins)'] = imdb_df['Duration (Mins)'].str.replace(r'[^\d.]+', '')
imdb_df['Ratings'] = imdb_df["Ratings"].astype(float)
imdb_df['Date Release'] = imdb_df["Date Release"].astype(int)
imdb_df['Duration (Mins)'] = imdb_df['Duration (Mins)'].astype(int)

In [147]:
imdb_df['Ratings'] = imdb_df["Ratings"].astype(float)
imdb_df['Date Release'] = imdb_df["Date Release"].astype(int)

In [148]:
scale_ratings = imdb_df["Ratings"]
scale_duration = imdb_df["Duration (Mins)"]
scale_ratings.max()

8.9

In [149]:
scaler = MinMaxScaler()
scale_ratings = scaler.fit_transform(imdb_df['Ratings'].values.reshape(-1,1))
scale_duration = scaler.fit_transform(imdb_df['Duration (Mins)'].values.reshape(-1,1))

In [150]:
imdb_df["MinMax Ratings"] = scale_ratings
imdb_df["MinMax Duration"] = scale_duration

In [151]:
mean_normalization_ratings = (scale_ratings - scale_ratings.mean()) / (scale_ratings.max() - scale_ratings.min())

In [152]:
imdb_df["NormalizedRatings"] = mean_normalization_ratings

In [153]:
imdb_df

,Movie Name,Date Release,Director,Duration (Mins),Ratings,Genre,Movie Stars,Filming Dates,Description,MinMax Ratings,MinMax Duration,NormalizedRatings
0,Gladiator,2000,Ridley Scott,155,8.5,"Action, Adventure, Drama","Russell Crowe, Joaquin Phoenix, Connie Nielsen...",18 January 1999 - 29 May 1999,A former Roman General sets out to exact venge...,0.857143,0.478571,0.264286
1,Inception,2010,Christopher Nolan,148,8.8,"Action, Adventure, Sci-Fi","Leonardo DiCaprio, Joseph Gordon-Levitt, Ellio...",19 June 2009 - 22 November 2009,A thief who steals corporate secrets through t...,0.964286,0.428571,0.371429
2,The Lord of the Rings: The Fellowship of the Ring,2001,Peter Jackson,178,8.8,"Action, Adventure, Drama","Elijah Wood, Ian McKellen, Orlando Bloom, Sean...",11 October 1999 - 22 December 2000,A meek Hobbit from the Shire and eight compani...,0.964286,0.642857,0.371429
3,The Lord of the Rings: The Two Towers,2002,Peter Jackson,179,8.7,"Action, Adventure, Drama","Elijah Wood, Ian McKellen, Viggo Mortensen, Or...",11 October 1999 - 22 December 2000,While Frodo and Sam edge closer to Mordor with...,0.928571,0.650000,0.335714
4,The Lord of the Rings: The Return of the King,2003,Peter Jackson,201,8.9,"Action, Adventure, Drama","Elijah Wood, Viggo Mortensen, Ian McKellen, Or...",11 October 1999 - 22 December 2000,Gandalf and Aragorn lead the World of Men agai...,1.000000,0.807143,0.407143
...,...,...,...,...,...,...,...,...,...,...,...,...
95,Zombieland,2009,Ruben Fleischer,88,7.6,"Adventure, Comedy, Fantasy","Jesse Eisenberg, Emma Stone, Woody Harrelson, ...",February 2009 - April 2009,A shy student trying to reach his family in Oh...,0.535714,0.000000,-0.057143
96,Speed,1994,Jan de Bont,116,7.2,"Action, Adventure, Thriller","Keanu Reeves, Dennis Hopper, Sandra Bullock, J...",7 September 1993 - 23 December 1993,A young police officer must prevent a bomb exp...,0.392857,0.200000,-0.200000
97,Jumanji,1995,Joe Johnston,104,7.0,"Adventure, Comedy, Family","Robin Williams, Kirsten Dunst, Bonnie Hunt, Jo...",14 November 1994 - 7 April 1995,When two kids find and play a magical board ga...,0.321429,0.114286,-0.271429
98,"The Chronicles of Narnia: The Lion, the Witch ...",2005,Andrew Adamson,143,6.9,"Adventure, Family, Fantasy","Tilda Swinton, Georgie Henley, William Moseley...",28 June 2004,Four kids travel through a wardrobe to the lan...,0.285714,0.392857,-0.307143
